In [1]:
import pandas as pd
import os
from datetime import datetime
from item_parser import parse
import sql_con

In [2]:
plytix_prods = pd.read_sql_query("""
    SELECT
        [Артикул],
        [Наименование], 
        [Brand]
    FROM [datahouse].[dbo].[View_Price1C_Auto]""", sql_con.engine_for_upload())
rhm_prods = pd.read_sql_query("""
   SELECT [sku]
      ,[label]
      ,[brand]
  FROM [datahouse].[dbo].[plytix_products]""", sql_con.engine_for_upload())
rhm_prods.columns = ['Артикул'] + list(rhm_prods.columns[1:])
merged_data = pd.merge(rhm_prods, plytix_prods, on='Артикул', how='inner')
merged_data.drop_duplicates(subset='Артикул', inplace=True)
merged_data.drop('Brand', axis=1, inplace=True)
merged_data.drop('label', axis=1, inplace=True)
merged_data = merged_data.reset_index(drop=True)

In [3]:
counter = 0
for item in merged_data["Наименование"][2500:]:
    errors_counter = 0
    while True:
        try:
            parse(item, clear = counter % 100 == 0)
            break
        except Exception as e:
            if errors_counter == 2:
                print(f'Два раза парсинг упал с ошибкой {item}')
                break
            errors_counter += 1
    counter += 1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Работа с позицией KAPOUS Маска Blond Bar с антижелтым эффектом, 500 мл
https://goldapple.ru/26872200002-blond-bar
17
Работа с позицией KAPOUS Шампунь Blond Bar с антижелтым эффектом, 500 мл
https://goldapple.ru/26872200003-blond-bar
18
Работа с позицией KAPOUS Мультиспрей для укладки волос 18 в 1 «Multi Spray», 250 мл
https://goldapple.ru/26871500012-multi-spray
19
Работа с позицией KAPOUS STUDIO Флюид для секущихся кончиков волос с маслами Авокадо и Оливы, 80 м
https://goldapple.ru/19000134377-macadamia-oil
20
21
22
23
24
25
26
Работа с позицией KAPOUS Питательный оттеночный бальзам для оттенков блонд, Перламутровый, 200 мл
https://goldapple.ru/26872700003-blond-bar
27
Работа с позицией KAPOUS Питательный оттеночный бальзам для оттенков блонд, Песочный, 200 мл
https://goldapple.ru/26872700005-blond-bar
28
Работа с позицией KAPOUS Питательный оттеночный бальзам для оттенков блонд, Серебро, 200 мл
https://goldapple.ru/19000002122-blond-bar-silver
2

KeyboardInterrupt: 

In [ ]:
counter

In [ ]:
merged_data.loc[merged_data['Наименование']=="DS Volume Mousse мусс для укладки сильной фиксации 300 мл"]

In [3]:
path_to_dir = r'C:/Users/admin/PycharmProjects/pythonProject/price_parsers/parsers_output/golden_apple/'
all_files = os.listdir(path_to_dir)
error_counter = 0
all_pages = []
for index, filename in enumerate(all_files):
    try: 
        all_pages.append(pd.read_excel(path_to_dir+filename))
    except Exception as e:
        raise(e)
        error_counter += 1

data = pd.concat(all_pages).drop(['Unnamed: 0'], axis=1)#.to_excel(f'{str(datetime.today().strftime("%Y-%m-%d"))}.xlsx',index=False)

In [4]:
data['Цена'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if cell_val is not None else None for cell_val in
                data['Цена']]
data['Цена со скидкой'] = [float(str(cell_val).replace('₽', '').replace(' ', '')) if not pd.isna(cell_val) else None for
                           cell_val in data['Цена со скидкой']]

In [5]:
merged_data.columns = list(merged_data.columns)[:-1]+['Название товара наше']

In [8]:
new_data = pd.merge(data, merged_data, on='Название товара наше')
new_data.drop('Бренд', axis=1)
new_data.to_excel('final_data_ga.xlsx')